# Caricamento dei dati

## Cosa impareremo

* I file CSV
* Caricare un file CSV direttamente da Internet
    * Dati internazionali dal repository Johns Hopkins CSSE
    * Dati italiani dal repository della Protezione Civile
* Visualizzare e manipolare le tabelle di dati
* Produrre semplici grafici dei dati

## Dati internazionali

Caricamento dei dati COVID internazionali.
I dati si trovano su github nel repository: https://github.com/CSSEGISandData/COVID-19

### Definizioni dei file

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

mondo_URL_base = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'
file_confermati = 'time_series_covid19_confirmed_global.csv'
file_morti = 'time_series_covid19_deaths_global.csv'
file_guariti = 'time_series_covid19_recovered_global.csv'

Questi file sono di tipo "CSV", che significa "Comma-Separated-Values", cioè valori separati da una virgola. È il modo più semplice ed universale per rappresentare dati tabulari come quelli che ci interessano, anche se non il pi compatto. Però ha il vantaggio di essere facilmente interpretabile dagli umani e non solo dalle macchine.

### Caricamento dati

Carichiamo ora i dati internazionali dei confermati.

In [ ]:
confermati_mondo = pd.read_csv(mondo_URL_base + file_confermati, index_col = 'Country/Region').sort_index()

In [ ]:
# Visualizzazione di un dataframe - utilizza il motore interno di rappresentazione
confermati_mondo

### Trattamento dei dati

Elaboriamo i dati in modo da ottenere solo le informazioni che ci interessano. In questo caso, rimuoviamo le colonne che non servono

In [ ]:
confermati_mondo.drop(columns = ['Province/State', 'Lat', 'Long'], inplace=True)

In [ ]:
confermati_mondo

### Estrazione dei dati per paese

Estraiamo adesso i dati del paese che ci interessa. La funzione ```loc``` permette di selezionare la riga (o le righe) del dataset che hanno un particolare indice.

In [ ]:
paese = 'Italy'
confermati_paese = confermati_mondo.loc[paese]
confermati_paese

Alcuni paesi come la Cina sono divisi in diverse province, quindi la stessa tecnica non dà il risultato desiderato.

In [ ]:
paese = 'China'
confermati_paese = confermati_mondo.loc[paese]
confermati_paese

Per ottenere il valore di tutta la Cina, dobbiamo sommare le varie righe tra di loro. Questo viene fatto con la funzione ```sum```

In [ ]:
paese = 'China'
confermati_paese = confermati_mondo.loc[paese].sum()
confermati_paese

Come vedete, otteniamo un risultato simile al precedente.

Possiamo già fare una prima visualizzazione di questi dati, semplicemente riportando su una linea l'andamento temporale dei casi confermati. Qua sotto mettiamo insieme i comandi precedenti. Modificando il valore della variabile ```paese``` possiamo ottenere grafici diversi.

In [ ]:
paese = 'China'
confermati_paese = confermati_mondo.loc[paese]

# per avere una implementazione generale, controlliamo se i dati riportati sono una lista o una tabella
if len(confermati_paese.shape) == 2:
    confermati_paese = confermati_paese.sum()

confermati_paese
plt.plot(confermati_paese)

Per rendere il risultato più generale, definiamo una funzione che ci permetta di evitare di scrivere ogni volta la sequenza di comandi precedente.

In [ ]:
def dati_paese(dati, paese):
    dati_uscita = dati.loc[paese]
    if len(dati_uscita.shape) == 2:
        dati_uscita = dati_uscita.sum()
    return dati_uscita

Così possiamo estrarre i dati per il paese che ci interessa utilizzando la funzione:

In [ ]:
confermati_italia = dati_paese(confermati_mondo, 'Italy')
confermati_cina = dati_paese(confermati_mondo, 'China')

plt.plot(confermati_italia, label='Italia')
plt.plot(confermati_cina, label='Cina')
plt.legend()

### Altri dati: morti e guarigioni

Possiamo adesso caricare i dati anche delle morti e delle guarigioni:

In [ ]:
morti_mondo = pd.read_csv(mondo_URL_base + file_morti, index_col = 'Country/Region').sort_index().drop(columns = ['Province/State', 'Lat', 'Long'])
guariti_mondo = pd.read_csv(mondo_URL_base + file_guariti, index_col = 'Country/Region').sort_index().drop(columns = ['Province/State', 'Lat', 'Long'])

Possiamo usare la funzione dati_paese definita prima anche per estrarre i dati da queste nuove tabelle

In [ ]:
morti_italia = dati_paese(morti_mondo, 'Italy')
morti_cina = dati_paese(morti_mondo, 'China')

guariti_italia = dati_paese(guariti_mondo, 'Italy')
guariti_cina = dati_paese(guariti_mondo, 'China')

Visualizziamo adesso i confermati, i morti e i guariti.

In [ ]:
plt.plot(confermati_italia, 'b', label='Confermati')
plt.plot(morti_italia, 'r', label='Morti')
plt.plot(guariti_italia, 'g', label='Guariti')
plt.legend()

In [ ]:
plt.plot(confermati_cina, 'b', label='Confermati')
plt.plot(morti_cina, 'r', label='Morti')
plt.plot(guariti_cina, 'g', label='Guariti')
plt.legend()

## Dati nazionali italiani

I dati nazionali italiani si trovano sul repository github della protezione civile: https://github.com/pcm-dpc/COVID-19

Nel nostro caso ci interessano i dati per regione. Il formato di questi dati è diverso dal formato di Johns Hopkins, visto che abbiamo un file CSV per ogni giorno con tutte le regioni e tutti i dati (confermati, morti, guarigioni) insieme.

Il nome di questi file è del tipo  ```dpc-covid19-ita-regioni-XXXXXXXX.csv``` dove le XX rappresentano una data. Per leggere tutti i file useremo un ciclo while e le funzioni datetime e timedelta di python.

In [ ]:
import datetime

italia_URL_base = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/'
formato_file_gioraliero = 'dpc-covid19-ita-regioni-%Y%m%d.csv'

data_inizio = datetime.date(2020,2,24) # inizio il 24 febbraio 2020

lista_dati = []

data_corrente = data_inizio
while data_corrente < datetime.date.today():
    file = data_corrente.strftime(formato_file_gioraliero)
    print(f'Lettura di {file}')
    lista_dati.append(pd.read_csv(italia_URL_base + file, index_col=['denominazione_regione', 'data']).sort_index())
    data_corrente += datetime.timedelta(days=1) # aggiunge un giorno alla data considerata
    
dati_italia_per_regione = pd.concat(lista_dati) # Aggrega i dati
print('Fatto')

Visualizziamo ora i dati aggregati.

In [ ]:
dati_italia_per_regione

Definiamo ora una funzione per ottenere dati simili a come ottenevamo con il dataset di prima.

In [ ]:
def dati_regione(regione):
    confermati = dati_italia_per_regione.loc[regione]['totale_casi'] # seleziona le righe "Regione" e la colonna "totale_casi"
    guariti = dati_italia_per_regione.loc[regione]['dimessi_guariti']
    morti = dati_italia_per_regione.loc[regione]['deceduti']
    return confermati, guariti, morti

Visualizziamo ora i dati per tre regioni italiane.

In [ ]:
confermati_puglia, guariti_puglia, morti_puglia = dati_regione('Puglia')
confermati_toscana, guariti_toscana, morti_toscana = dati_regione('Toscana')
confermati_lombardia, guariti_lombardia, morti_lombardia = dati_regione('Lombardia')

plt.plot(confermati_puglia, label='Puglia')
plt.plot(confermati_toscana, label='Toscana')
plt.plot(confermati_lombardia, label='Lombardia')
plt.legend()


# Cosa abbiamo imparato

* Struttura dei file CSV
* Leggere i file CSV con pandas
* Manipolare le tabelle di pandas (ordinare i dati, rimuovere colonne con ```drop```)
* Estrarre dati da una tabella con ```loc[]```
* Visualizzare i dati con ```plt.plot()```
* Leggere dati da file multiple in maniera automatizzata
* Aggregare varie tabelle di dati

# La prossima settimana

* Manipolazione dei dati (calcolo delle variazioni)
* Visualizzazioni avanzate (scala logaritmica, diagrammi a barre, altre rappresentazioni)

# Autore

Copyright Francesco Santini <francesco.santini@gmail.com> 2020 - Rilasciato sotto GNU General Public License v3.0 o successive